# Louvain Community Detection


In this notebook, we will use cuGraph to identify the cluster in a test graph using the Louvain algorithm  

Notebook Credits
* Original Authors: Bradley Rees and James Wyles
* Created:   08/01/2019
* Last Edit: 10/16/2019

RAPIDS Versions: 0.10.0

Test Hardware
* GV100 32G, CUDA 10.0



## Introduction

The Louvain method of community detection is a greedy heirarical clsutering algorithm which seeks to optimize Modularity as it progresses. Louvain starts with each vertex in its own clusters and iteratively merges groups using graph contraction.  

For a detailed description of the algorithm see: https://en.wikipedia.org/wiki/Louvain_Modularity

It takes as input a cugraph.Graph object and returns as output a 
cudf.Dataframe object with the id and assigned partition for each 
vertex as well as the final modularity score


To compute the Louvain cluster in cuGraph use: <br>
* __df = cugraph.louvain(G)__
    * G: A cugraph.Graph object

Returns:
* tupal __dataframe__ and __modularity__

    * __df__: cudf.DataFrame with two names columns:
        * louvain["vertex"]: The vertex id.
        * louvain["partition"]: The assigned partition.
    
    * __modularity__ : the overall modularity of the graph

All vertices with the same partition ID are in the same cluster


#### Note
Parallel Louvain produces different modularity scores that seriel Louvain.  A complete technical write-up is being produced and will be linked here when available.  

### References

* Blondel, V. D., Guillaume, J.-L., Lambiotte, R., and Lefebvre, E. Fast unfolding of communities in large networks. Journal of statistical mechanics: theory and experiment 2008, 10 (2008), P10008.


## cuGraph Notice 
The current version of cuGraph has some limitations:

* Vertex IDs need to be 32-bit integers.
* Vertex IDs are expected to be contiguous integers starting from 0.

cuGraph provides the renumber function to mitigate this problem. Input vertex IDs for the renumber function can be either 32-bit or 64-bit integers, can be non-contiguous, and can start from an arbitrary number. The renumber function maps the provided input vertex IDs to 32-bit contiguous integers starting from 0. cuGraph still requires the renumbered vertex IDs to be representable in 32-bit integers. These limitations are being addressed and will be fixed soon.    

### Test Data
We will be using the Zachary Karate club dataset 
*W. W. Zachary, An information flow model for conflict and fission in small groups, Journal of
Anthropological Research 33, 452-473 (1977).*


![Karate Club](../img/zachary_black_lines.png)


### Prep

In [1]:
# Import needed libraries
import cugraph
import cudf

## Read data using cuDF

In [2]:
# Test file    
datafile='../data//karate-data.csv'

In [3]:
# read the data using cuDF
gdf = cudf.read_csv(datafile, delimiter='\t', names=['src', 'dst'], dtype=['int32', 'int32'] )

In [4]:
# Since our data set start numbering at "1" the code will add a disconnected vertex with ID of "0".  While that will not cause the analytic 
# to crash, it will produce one additional cluster.  So let's just renumber everything
# Rather than using the renumbering function, we will just subtract 1 from every ID
gdf["src_0"] = gdf["src"] - 1
gdf["dst_0"] = gdf["dst"] - 1

In [5]:
# The algorithm also requires that there are vertex weights.  Just use 1.0 
gdf["data"] = 1.0

In [6]:
# just for fun, let's look at the data types in the dataframe
gdf.dtypes

src        int32
dst        int32
src_0      int32
dst_0      int32
data     float64
dtype: object

In [7]:
# create a Graph 
G = cugraph.Graph()
G.from_cudf_edgelist(gdf, source='src_0', destination='dst_0', edge_attr='data')

In [8]:
# Call Louvain on the graph
df, mod = cugraph.louvain(G) 

In [9]:
# Print the modularity score
print('Modularity was {}'.format(mod))
print()

Modularity was 0.4027777777777778



In [10]:
df.dtypes

vertex       int32
partition    int32
dtype: object

In [11]:
# How many partitions where found
part_ids = df["partition"].unique()

In [12]:
print(str(len(part_ids)) + " partition detected")

4 partition detected


In [13]:
# print the clusters.  ADD a +1 to vertex IDs to match picture
for p in range(len(part_ids)):
    part = []
    for i in range(len(df)):
        if (df['partition'][i] == p):
            part.append(df['vertex'][i] +1)
    print("Partition " + str(p) + ":")
    print(part)


Partition 0:
[1, 2, 3, 4, 8, 10, 12, 13, 14, 18, 20, 22]
Partition 1:
[5, 6, 7, 11, 17]
Partition 2:
[9, 15, 16, 19, 21, 23, 27, 29, 30, 31, 32, 33, 34]
Partition 3:
[24, 25, 26, 28]


___
Copyright (c) 2019, NVIDIA CORPORATION.

Licensed under the Apache License, Version 2.0 (the "License");  you may not use this file except in compliance with the License. You may obtain a copy of the License at http://www.apache.org/licenses/LICENSE-2.0

Unless required by applicable law or agreed to in writing, software distributed under the License is distributed on an "AS IS" BASIS, WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied. See the License for the specific language governing permissions and limitations under the License.
___